In [ ]:
from pathlib import Path
import re
from shutil import copy

from lotr.file_utils import mirror_fish_folder, folder_2_fid
from tqdm import tqdm
import shutil

### Cleanup folders from E0040 to E0071

In [ ]:
master_source = Path(r"\\FUNES\Shared\experiments\E0040_motions_cardinal")
dest_master_folder = Path(r"\\FUNES\Shared\experiments\E0071_lotr")

In [ ]:
all_folders_tomove = [f for f in master_source.glob("*/*[0-9]_f[0-9]*") if f.is_dir()]

In [ ]:
for f in tqdm(all_folders_tomove):
    parent_dest_name = dest_master_folder / folder_2_fid(f)
    parent_dest_name.mkdir(exist_ok=True)
    
    dest_name = parent_dest_name / f.name
    dest_name.mkdir(exist_ok=True)
    
    mirror_fish_folder(
            f, dest_name,
            file_patterns=[
                    "*behavior_log*",
                    "*stimulus_log*",
                    "*metadata.json",
                    "data_from_suite2p_unfiltered.h5",
                    "*video.mp4",
                    "*video_times.hdf5",
                    "*img.png",
                    "*select*"
                ]
        )

In [ ]:
for f in tqdm(all_folders_tomove):
    fid = "".join(re.split(r"(\d+)", f.name.split("_")[1])[:2])  # take only actuall fish id of date and fish number
    date = f.name.split("_")[0]
    parent_dest_name = dest_master_folder / f"{date}_{fid}"
    
    print(parent_dest_name)

### Move selected.h5 files from local to FUNES

In [ ]:
local_all_data = Path("/Users/luigipetrucco/Desktop/all_source_data")
dest_master_folder = Path(r"/Volumes/Shared/experiments/E0071_lotr")

In [ ]:
all_folders_tomove = [f.parent for f in local_all_data.glob("*/*[0-9]_f[0-9]*/selected.h5")]

In [ ]:
for f in tqdm(all_folders_tomove):
    # print(f.name)
    
    parent_dest_name = dest_master_folder / folder_2_fid(f)
    # print(parent_dest_name)
    # parent_dest_name.mkdir(exist_ok=True)
    
    dest_name = parent_dest_name / f.name
    # dest_name.mkdir(exist_ok=True)
    if dest_name.exists():
        mirror_fish_folder(
                f, dest_name,
                file_patterns=[
                        "*select*.h5"
                    ]
            )
        print("moving ", f)

### Move valid fish to ring folder

In [ ]:
dest_master_folder = Path("/Users/luigipetrucco/Desktop/all_source_data")
all_valid = set([f.parent for f in dest_master_folder.glob("*/*") if len(list(f.glob("*selected*"))) > 0])
dest_valid = Path("/Users/luigipetrucco/Desktop/all_source_data/full_ring")

In [ ]:
len(all_valid)

In [ ]:
for f in (all_valid):
    # print(dest_valid / f.name)
    #if len(list(f.glob("*selected*"))) > 0:
    #    print(f.parent, f.parent.parent / full_ring / f.parent.name)
    shutil.move(f, dest_valid / f.name)

### Restructure folder of raw acquisitions

In [ ]:
master_folder = Path("/Users/luigipetrucco/Desktop/all_source_data")
dest_folder = Path("/Users/luigipetrucco/Desktop/all_source_data")

In [ ]:
for f in list(master_folder.glob("*[0-9]_f[0-9]*")):
    fid = folder_2_fid(f)
    
    dest_fid_folder = dest_folder / fid
    dest_fid_folder.mkdir(exist_ok=True)
    
    shutil.move(f, dest_fid_folder / f.name)

### Mirror files on local and FUNES

In [ ]:
funes_folder = Path("/Volumes/Shared/experiments/E0071_lotr")
local_folder = Path("/Users/luigipetrucco/Desktop/all_source_data")

In [ ]:
funes_dirs = set([f.name for f in (funes_folder / "full_ring").glob("*[0-9]_f[0-9]*")])
local_dirs = set([f.name for f in (local_folder / "full_ring").glob("*[0-9]_f[0-9]*")])

In [ ]:
file_patterns=[
                    "*behavior_log*",
                    "*stimulus_log*",
                    "*metadata.json",
                    "data_from_suite2p_unfiltered.h5",
                    "*img.png",
                    "*select*"
                ]

for fid in tqdm(list(funes_dirs - local_dirs)):
    dest_parent = local_folder / "full_ring" / fid
    dest_parent.mkdir(exist_ok=True)
    
    for subf in (funes_folder / "full_ring" / fid).glob("*[0-9]_f[0-9]*"):
        mirror_fish_folder(
                    subf, dest_parent / subf.name,
                    file_patterns=file_patterns
                )